## Escenario de pagos bancarios

Un banco le ha pedido que desarrolle un nuevo sistema backend para gestionar sus pagos programados.

Se trata principalmente de una carga de trabajo OLTP (Procesamiento de Transacciones En Línea) con procesos por lotes diarios. Los elementos de la(s) tabla(s) representan pagos programados entre cuentas. A medida que se insertan las partidas, se programan en una fecha específica para que se procese el pago. Cada día, las partidas se envían regularmente a un sistema transaccional para su procesamiento, momento en el que su estado cambia a `PENDING`. Cuando la transacción se realiza correctamente, el estado de la partida pasa a `PROCESSED` y se actualiza con un nuevo ID de transacción.

Dimensiones de la carga de trabajo:

* Las cuentas pueden tener varios pagos programados para cualquier día en el futuro.
* Los pagos tienen los siguientes campos de datos: AccountID, ScheduledDate, Status (`SCHEDULED`, `PENDING` o `PROCESSED`), DataBlob (el tamaño total del artículo es <= 8 KB)
* Un millón de pagos programados automatizados se añaden cada día a la 1:00 AM para ese día, que deben completarse en 30 minutos.
* Cada día se añaden un millón de pagos con el estado `SCHEDULED`, principalmente en el horario de 6 AM a 6 PM.
* Durante el día, se ejecuta regularmente un trabajo por lotes para consultar los pagos `SCHEDULED` de hoy. Este servicio envía las partidas `SCHEDULED` al servicio de transacciones. Al enviar las partidas al servicio de transacciones, el estado del pago cambia a `PENDING`.
* Cuando el servicio de transacciones finaliza, el estado de un artículo cambia a `PROCESSED` y se añade un nuevo ID de transacción al artículo.
* Es necesario devolver artículos para una cuenta específica cuyo pago está programado para los próximos 90 días.
* El servicio transaccional tiene que recuperar todos los artículos para una fecha específica (por ejemplo, hoy) en todas las cuentas. Tiene que ser capaz de recuperar artículos que estén específicamente `SCHEDULED` o `PENDING`.

**Su reto:** Desarrollar un modelo de datos NoSQL para el banco que cumpla los requisitos de pagos programados.

**Desafío extra:** Al final de cada día, todas las partidas que fueron `PROCESSED` deben trasladarse a una tabla a largo plazo (debido al cumplimiento, los datos deben estar en una tabla separada). Diseñe un segundo modelo de datos que cumpla los mismos requisitos de acceso que el anterior y añada otro requisito para devolver un artículo específico asociado a un ID de transacción.

#### Patrones de acceso

*Los patrones de acceso en el escenario son los siguientes:*

* Insertar pagos programados.
* Devolver pagos programados por usuario para los próximos 90 días.
* Devolver pagos de todos los usuarios para una fecha específica por estado (`SCHEDULED` o `PENDING`).

*Identificar posibles claves de partición para cumplir con el patrón de acceso primario:*

* ¿Qué atributo de elemento (AccountID, ScheduledDate, Status, DataBlob) se ajusta a los patrones de acceso?
* ¿Cuál es la organización natural de las partidas de pago relacionadas (para devolver las partidas recogidas en relación con los patrones de acceso anteriores)?
* Considere la dimensión del acceso: tanto lecturas como escrituras.

*A la hora de determinar cómo organizar los elementos relacionados con el patrón de acceso principal:*

* ¿Con qué organización deben escribirse los elementos para devolver los elementos por usuario para un intervalo de fechas (ordenar por)?
* ¿Cuál es la jerarquía de las relaciones y cuándo se aplica (de más general a más específico)?

*Cumplimiento del tercer patrón de acceso:*

* El tercer patrón de acceso es OLTP y puede cumplirse directamente en DynamoDB

In [ ]:
import boto3
import random
import uuid
from botocore.exceptions import ClientError
import pandas as pd
from spdynamodb import DynamoTable
import json
import time
from decimal import Decimal
from datetime import datetime, timedelta

client = boto3.client("dynamodb", region_name="us-east-1")

In [ ]:
#dt = DynamoTable(profile_name='089715336747_DynamoAttributes')
dt=DynamoTable()
try:
    dt.select_table('BankPayments')
    print(dt)
except:
    dt.create_table(
        table_name='BankPayments',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
    )

In [ ]:
# Create Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    sort_index="GSI1SK",
    sort_type="S",
    i_name="GSI1"
)

In [ ]:
status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

### Insertar pagos programados

In [ ]:
# FUNCTION TO GENERATE DATA
import json
def generate_data(total_items=300):
    count_total = 0
    pk, sk, AccountId, ScheduledDate, Status, DataBlob, TransactionId, GSI1PK, GSI1SK, GSI2PK, GSI2SK, TransactionDate = [], [], [], [], [], [], [], [], [], [], [], []
    for i in range(total_items):
        account = str(random.randint(10000000000, 99999999999))
        for m in range(1, random.randint(2, 25)):
            if count_total == total_items:
                break
            uid = str(uuid.uuid4()).split("-")[0]
            # Get the current date and time
            iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'
            # Set the date range
            start_date = datetime(2023, 6, 1)
            end_date = datetime(2024, 6, 30)
            # Generate a random number of days to add to the start date
            random_days = random.randint(0, (end_date - start_date).days)
            # Calculate the random date
            random_date = start_date + timedelta(days=random_days)
            # Format the date as a string
            random_date_str = random_date.strftime("%Y-%m-%d")
            scheduled_date_ns = str(int(random_date.timestamp())) + str(random.randint(111111111, 999999999))
            # === Append data to lists =====
            pk.append(account+"#PAYMENT")
            sk.append("SCHEDULED#"+scheduled_date_ns)
            AccountId.append(account)
            ScheduledDate.append(random_date_str)
            Status.append("SCHEDULED")
            DataBlob.append({"Amount": random.randint(1, 10000), "Currency": "USD", "BeneficiaryId": str(random.randint(10000000000, 99999999999))})
            TransactionId.append(uid)
            GSI1PK.append("SCHEDULED#" + str(random.randint(1, 5)))
            GSI1SK.append(random_date_str)
            TransactionDate.append(iso_date)
            count_total += 1
    df_main = pd.DataFrame(
        {
            "PK": pk,
            "SK": sk,
            "AccountId": AccountId,
            "ScheduledDate": ScheduledDate,
            "Status": Status,
            "DataBlob": DataBlob,
            "TransactionId": TransactionId,
            "GSI1PK": GSI1PK,
            "GSI1SK": GSI1SK,
            "TransactionDate": TransactionDate
        }
    )   
    return df_main

In [ ]:
df = generate_data(total_items=2500)
dt.batch_pandas(df)

#### Devolver pagos programados por usuario para los próximos 90 días

In [ ]:
account_id = list(df['PK'].value_counts().to_dict().items())[0][0]
now_time = datetime.now()
delta_date = now_time + timedelta(days=90)
now_time_unix = str(int(now_time.timestamp()))
delta_time_unix = str(int(delta_date.timestamp()))

In [ ]:
query = f"SCHEDULED#{now_time_unix}_SCHEDULED#{delta_time_unix}"
response = dt.query(pk_value=account_id, sk_value=query)

In [ ]:
print("Total items:", len(response))
print("==============")
for item in response:
    print(item['PK'], item['SK'], item['DataBlob'], "- Scheduled Date:", item['ScheduledDate'])

#### Devolver pagos de todos los usuarios para una fecha específica por estado (SCHEDULED o PENDING)

In [ ]:
def create_query_input(gsi_id, select_date, between=False):
    if between:
        return {
            "TableName": "BankPayments",
            "IndexName": "GSI1",
            "KeyConditionExpression": "#3e1e0 = :3e1e0 And #3e1e1 BETWEEN :3e1e1 AND :3e1e2",
            "ExpressionAttributeNames": {"#3e1e0":"GSI1PK","#3e1e1":"GSI1SK"},
            "ExpressionAttributeValues": {":3e1e0": {"S":gsi_id},":3e1e1": {"S":select_date[0]},":3e1e2": {"S":select_date[1]}}
        }
    else:
        return {
            "TableName": "BankPayments",
            "IndexName": "GSI1",
            "KeyConditionExpression": "#3e1e0 = :3e1e0 And #3e1e1 = :3e1e1",
            "ExpressionAttributeNames": {"#3e1e0":"GSI1PK","#3e1e1":"GSI1SK"},
            "ExpressionAttributeValues": {":3e1e0": {"S":gsi_id},":3e1e1": {"S":select_date}}
        }

def get_items(select_date, gsi_id):
    """
    Get items from DynamoDB table using GSI1
    : select_date: Date to query. If the date is between two dates, pass a list with two dates
    : gsi_id: GSI1PK value
    """
    all_items = []
    
    if gsi_id == "SCHEDULED":
        for i in range(1, 6):
            gsi_id = f"SCHEDULED#{i}"
            between = True if type(select_date) == list else False
            input_data = create_query_input(gsi_id, select_date, between)
            try:
                response = client.query(**input_data)
                all_items.append(response['Items'])
            except ClientError as e:
                print(f"Error:", e)
        all_items = [x for y in all_items for x in y]
    else:
        between = True if type(select_date) == list else False
        input_data = create_query_input(gsi_id, select_date, between)
        try:
            response = client.query(**input_data)
            all_items = response['Items']
        except ClientError as e:
            print(f"Error:", e)
    return all_items

In [ ]:
all_items = get_items(select_date="2023-06-21", gsi_id="SCHEDULED")   
print("Total items:", len(all_items))
print("==============")
for item in all_items:
    print(item['PK'], item['SK'], item['DataBlob'], "- Scheduled Date:", item['ScheduledDate'])

#### Cambiar estado a pendiente para pagos programados para el día de hoy


In [40]:
#today = datetime.now().strftime("%Y-%m-%d")
today = "2023-06-28"
all_items = get_items(select_date=today, gsi_id="SCHEDULED")
print("Total items: ", len(all_items))

Total items:  8


In [41]:
transact_items = []
for item in all_items:
    new_sk = item['SK']['S'].replace("SCHEDULED", "PENDING")
    transact = [
        {
            'Put': {
            'TableName': dt.table_name,
                'Item': {
                        'PK': {'S': item['PK']['S']},
                        'SK': {'S': new_sk},
                        'AccountId': {'S': item['AccountId']['S']},
                        'ScheduledDate': {'S': item['ScheduledDate']['S']},
                        'Status': {'S': 'PENDING'},
                        'DataBlob': {'M': item['DataBlob']['M']},
                        'TransactionId': {'S': item['TransactionId']['S']},
                        'GSI1PK': {'S': 'PENDING'},
                        'GSI1SK': {'S': item['GSI1SK']['S']},
                        'TransactionDate': {'S': item['TransactionDate']['S']}
                    }
            }
        },
        {
            'Delete': {
                'TableName': dt.table_name,
                'Key': {
                    'PK': {'S': item['PK']['S']},
                    'SK': {'S': item['SK']['S']}
                }
            }
        }
    ]
    transact_items.append(transact)
transact_items = [x for y in transact_items for x in y]

try:
    response = client.transact_write_items(
        TransactItems=transact_items,
        ReturnConsumedCapacity='TOTAL'
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print(f"Success! - ConsumedCapacity: {response['ConsumedCapacity']}")    

Success! - ConsumedCapacity: [{'TableName': 'BankPayments', 'CapacityUnits': 64.0, 'WriteCapacityUnits': 64.0}]


In [42]:
all_items = get_items(select_date=["2023-06-21", "2023-09-22"], gsi_id="PENDING")
print("Total items:", len(all_items))
print("==============")
for item in all_items:
    print(item['PK']['S'], item['SK']['S'], item['DataBlob'], "- Status:", item['Status']['S'])

Total items: 8
{'S': '97794918497#PAYMENT'} {'S': 'PENDING#1687921200169250281'} {'M': {'BeneficiaryId': {'S': '10474928557'}, 'Amount': {'N': '5888'}, 'Currency': {'S': 'USD'}}} - Status: PENDING
{'S': '51562874714#PAYMENT'} {'S': 'PENDING#1687921200915195778'} {'M': {'BeneficiaryId': {'S': '82351126355'}, 'Amount': {'N': '9739'}, 'Currency': {'S': 'USD'}}} - Status: PENDING
{'S': '79271622392#PAYMENT'} {'S': 'PENDING#1687921200918568865'} {'M': {'BeneficiaryId': {'S': '54105439722'}, 'Amount': {'N': '7700'}, 'Currency': {'S': 'USD'}}} - Status: PENDING
{'S': '91132915947#PAYMENT'} {'S': 'PENDING#1687921200500675399'} {'M': {'BeneficiaryId': {'S': '79613081313'}, 'Amount': {'N': '6512'}, 'Currency': {'S': 'USD'}}} - Status: PENDING
{'S': '62693542379#PAYMENT'} {'S': 'PENDING#1687921200278195558'} {'M': {'BeneficiaryId': {'S': '95884816769'}, 'Amount': {'N': '3047'}, 'Currency': {'S': 'USD'}}} - Status: PENDING
{'S': '61678675204#PAYMENT'} {'S': 'PENDING#1687921200597290684'} {'M': {'B

#### Procesar pagos pendientes

In [44]:
all_items = get_items(select_date=today, gsi_id="PENDING")
print("Total items:", len(all_items))

Total items: 8


In [45]:
transact_items = []
for item in all_items:
    new_sk = item['SK']['S'].replace("PENDING", "PROCESSED")
    uid = str(uuid.uuid4())
    iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'
    transact = [
        {
            'Put': {
            'TableName': dt.table_name,
                'Item': {
                        'PK': {'S': item['PK']['S']},
                        'SK': {'S': new_sk},
                        'AccountId': {'S': item['AccountId']['S']},
                        'ScheduledDate': {'S': item['ScheduledDate']['S']},
                        'Status': {'S': 'PROCESSED'},
                        'DataBlob': {'M': item['DataBlob']['M']},
                        'TransactionId': {'S': uid},
                        'GSI1PK': {'S': 'PROCESSED'},
                        'GSI1SK': {'S': item['GSI1SK']['S']},
                        'TransactionDate': {'S': iso_date}
                    }
            }
        },
        {
            'Delete': {
                'TableName': dt.table_name,
                'Key': {
                    'PK': {'S': item['PK']['S']},
                    'SK': {'S': item['SK']['S']}
                }
            }
        }
    ]
    transact_items.append(transact)
transact_items = [x for y in transact_items for x in y]

try:
    response = client.transact_write_items(
        TransactItems=transact_items,
        ReturnConsumedCapacity='TOTAL'
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print(f"Success! - ConsumedCapacity: {response['ConsumedCapacity']}")  

Success! - ConsumedCapacity: [{'TableName': 'BankPayments', 'CapacityUnits': 64.0, 'WriteCapacityUnits': 64.0}]


In [46]:
all_items = get_items(select_date=today, gsi_id="PROCESSED")
print("Total items:", len(all_items))
print("==============")
for item in all_items:
    print(item['PK']['S'], item['SK']['S'], item['DataBlob'], "- Status:", item['Status']['S'])

Total items: 8
{'S': '91132915947#PAYMENT'} {'S': 'PROCESSED#1687921200500675399'} {'M': {'BeneficiaryId': {'S': '79613081313'}, 'Amount': {'N': '6512'}, 'Currency': {'S': 'USD'}}} - Status: PROCESSED
{'S': '79271622392#PAYMENT'} {'S': 'PROCESSED#1687921200918568865'} {'M': {'BeneficiaryId': {'S': '54105439722'}, 'Amount': {'N': '7700'}, 'Currency': {'S': 'USD'}}} - Status: PROCESSED
{'S': '62693542379#PAYMENT'} {'S': 'PROCESSED#1687921200278195558'} {'M': {'BeneficiaryId': {'S': '95884816769'}, 'Amount': {'N': '3047'}, 'Currency': {'S': 'USD'}}} - Status: PROCESSED
{'S': '61678675204#PAYMENT'} {'S': 'PROCESSED#1687921200597290684'} {'M': {'BeneficiaryId': {'S': '67191927031'}, 'Amount': {'N': '1968'}, 'Currency': {'S': 'USD'}}} - Status: PROCESSED
{'S': '63940110945#PAYMENT'} {'S': 'PROCESSED#1687921200511351367'} {'M': {'BeneficiaryId': {'S': '21641302033'}, 'Amount': {'N': '4526'}, 'Currency': {'S': 'USD'}}} - Status: PROCESSED
{'S': '22853057068#PAYMENT'} {'S': 'PROCESSED#168792120

#### Traladar los pagos procesados a una tabla a largo plazo

In [47]:
dt_processed=DynamoTable()
try:
    dt_processed.select_table('BankPaymentsProcessed')
    print(dt)
except:
    dt_processed.create_table(
        table_name='BankPaymentsProcessed',
        partition_key='id',
        partition_key_type='S'
    )

- Table name: BankPayments            
- Table arn: arn:aws:dynamodb:us-east-1:279069313747:table/BankPayments            
- Table creation: 2023-06-21 10:44:46            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'GSI1PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI1SK', 'AttributeType': 'S'}, {'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [48]:
all_items = get_items(select_date=today, gsi_id="PROCESSED")
print("Total items:", len(all_items))

Total items: 8


In [50]:
transact_items = []
for item in all_items:
    iso_date = datetime.utcnow().isoformat().split('.')[0]+'Z'
    transact = [
        {
            'Put': {
            'TableName': dt_processed.table_name,
                'Item': {
                        'id': {'S': item['TransactionId']['S']},
                        'AccountId': {'S': item['AccountId']['S']},
                        'ScheduledDate': {'S': item['ScheduledDate']['S']},
                        'Status': {'S': item['Status']['S']},
                        'DataBlob': {'M': item['DataBlob']['M']},
                        'TransactionDate': {'S': item['TransactionDate']['S']}
                    }
            }
        },
        {
            'Update': {
                'TableName': dt.table_name,
                'Key': {
                    'PK': {'S': item['PK']['S']},
                    'SK': {'S': item['SK']['S']}
                },
                'UpdateExpression': 'SET #c = :st, #dc = :dc',
                'ExpressionAttributeNames': {'#c': 'Status', '#dc': 'GSI1PK'},
                'ExpressionAttributeValues': {':st': {'S': 'ARCHIVED'}, ':dc': {'S': 'ARCHIVED'}}
            }
        }
    ]
    transact_items.append(transact)
transact_items = [x for y in transact_items for x in y]

try:
    response = client.transact_write_items(
        TransactItems=transact_items,
        ReturnConsumedCapacity='TOTAL'
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print(f"Success! - ConsumedCapacity: {response['ConsumedCapacity']}")  

Success! - ConsumedCapacity: [{'TableName': 'BankPayments', 'CapacityUnits': 48.0, 'WriteCapacityUnits': 48.0}, {'TableName': 'BankPaymentsProcessed', 'CapacityUnits': 16.0, 'WriteCapacityUnits': 16.0}]


In [53]:
all_items = get_items(select_date=today, gsi_id="ARCHIVED")
print("Total items:", len(all_items))
print("==============")
for item in all_items:
    print(item['PK']['S'], item['TransactionId']['S'], item['DataBlob'], "- Status:", item['Status']['S'])

Total items: 8
91132915947#PAYMENT 649002e8-9904-4963-9c8a-7a6fa94b73ba {'M': {'BeneficiaryId': {'S': '79613081313'}, 'Amount': {'N': '6512'}, 'Currency': {'S': 'USD'}}} - Status: ARCHIVED
79271622392#PAYMENT d1239f97-c9bd-4631-9078-9020a2f6b816 {'M': {'BeneficiaryId': {'S': '54105439722'}, 'Amount': {'N': '7700'}, 'Currency': {'S': 'USD'}}} - Status: ARCHIVED
62693542379#PAYMENT 85c488a8-bc29-4c8c-a32c-8446c89d1eac {'M': {'BeneficiaryId': {'S': '95884816769'}, 'Amount': {'N': '3047'}, 'Currency': {'S': 'USD'}}} - Status: ARCHIVED
61678675204#PAYMENT ff8ae03f-0e86-4a89-906e-4a0461d3b9d5 {'M': {'BeneficiaryId': {'S': '67191927031'}, 'Amount': {'N': '1968'}, 'Currency': {'S': 'USD'}}} - Status: ARCHIVED
63940110945#PAYMENT ea553308-5240-4b62-aedf-f03b82f781ca {'M': {'BeneficiaryId': {'S': '21641302033'}, 'Amount': {'N': '4526'}, 'Currency': {'S': 'USD'}}} - Status: ARCHIVED
22853057068#PAYMENT 5b822cda-2b63-4348-94b1-2c627f297eab {'M': {'BeneficiaryId': {'S': '49237130584'}, 'Amount': {'